In [1]:
#!pip install wikipedia langchain BeautifulSoup


In [1]:
from langchain import OpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.agents import Tool, tool
from langchain import agents
from bs4 import BeautifulSoup
import langchain
langchain.debug = False
import os
import requests

os.environ['OPENAI_API_KEY'] = 'YOUR_VALID_KEY'

In [5]:
import requests
name = 'methanol'
r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/' + name +'/cids/JSON')
response = r.json()
cid = response["IdentifierList"]["CID"][0]

In [6]:
data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/xml")
html = BeautifulSoup(data.content, "xml")
bp_tag = html.find(name='TOCHeading', string='Boiling Point')
bp_tag.find_next_sibling('Information').find(name='String').string

'148.3 °F at 760 mmHg (NTP, 1992)'

In [7]:
safety_tag = html.find(name='TOCHeading', string='Health Hazards')
safety_tag.find_next_sibling('Information').find(name='String').string

'Exposure to excessive vapor causes eye irritation, head- ache, fatigue and drowsiness. High concentrations can produce central nervous system depression and optic nerve damage. 50,000 ppm will probably cause death in 1 to 2 hrs. Can be absorbed through skin. Swallowing may cause death or eye damage. (USCG, 1999)'

In [8]:
def find_property(compound, property):
    '''This function queries the compound from pubchecm and obtains property of the compund.
    example use: find_property('KOH', 'Melting Point) '''
    try:
        # query the PubChem database
        r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/' + compound +'/cids/JSON')
        response = r.json()
        cid = response["IdentifierList"]["CID"][0]
        data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/xml")
        html = BeautifulSoup(data.content, "xml")
        tag = html.find(name='TOCHeading', string=property.title()).find_next_sibling('Information').find(name='String').string
        return tag
    except:
        print(f"Could not find the {property} for {compound} on PubChem.")

from langchain.tools import StructuredTool
tools = [StructuredTool.from_function(find_property)]

In [60]:
from langchain.memory import ConversationBufferMemory
llm = OpenAI(
        model_name='gpt-4',
        temperature=0,
    )

memory = ConversationBufferMemory()
agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True, memory=memory,  
    )

In [61]:
agent.run(input="Experiment: Heat up methanol up to its boiling point. Check for Health Hazards of the compound and stop the experiment if unsafe.")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "find_property",
  "action_input": {
    "compound": "methanol",
    "property": "Health Hazards"
  }
}
```
Observation: Exposure to excessive vapor causes eye irritation, head- ache, fatigue and drowsiness. High concentrations can produce central nervous system depression and optic nerve damage. 50,000 ppm will probably cause death in 1 to 2 hrs. Can be absorbed through skin. Swallowing may cause death or eye damage. (USCG, 1999)
Thought:The health hazards of methanol are quite severe. Exposure can cause eye irritation, headaches, fatigue, and drowsiness. High concentrations can lead to central nervous system depression and optic nerve damage. It can be fatal in high concentrations and can be absorbed through the skin. Swallowing methanol can also cause death or eye damage. Given these hazards, it would be unsafe to proceed with the experiment without proper safety measures. Now, I will find the boiling point of methano

'Methanol has a boiling point of 148.3 °F at 760 mmHg. However, it has severe health hazards. Exposure can cause eye irritation, headaches, fatigue, and drowsiness. High concentrations can lead to central nervous system depression and optic nerve damage. It can be fatal in high concentrations and can be absorbed through the skin. Swallowing methanol can also cause death or eye damage. Given these hazards, it would be unsafe to proceed with the experiment without proper safety measures.'